In [194]:
import numpy as np
import timeit

In [195]:
m = 2.5
k = 8

In [196]:
def generate_matrix_and_vector(n, dtype=np.float64):
    matrix = np.zeros((n,n), dtype=dtype)

    for i in range(1, n+1):
        for j in range(1, n+1):
            if i == j:
                matrix[i-1, j-1] = k
            elif j > i:
                matrix[i-1, j-1] = (-1) ** j * (m / j)
            elif j == i-1:
                matrix[i-1, j-1] = m/i


    vector = np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=dtype)

    return matrix, vector

In [197]:
def difference(v1, v2):
    differences = np.abs(v1 - v2)

    max_difference = np.max(differences)
    return max_difference

In [198]:
def jacobi_iteration_matrix(A):
    D = np.diag(np.diag(A))
    D_inv = np.linalg.inv(D)
    M = np.eye(A.shape[0]) - D_inv @ A
    return M

def spectral_radius(matrix):
    eigenvalues = np.linalg.eigvals(matrix)
    return max(abs(eigenvalues))

In [199]:
for i in range(3, 200):
    A, _ = generate_matrix_and_vector(i)
    M = jacobi_iteration_matrix(A)
    rho = spectral_radius(M)
    print(f"n = {i}, rho = {rho}")

n = 3, rho = 0.15625000000000003
n = 4, rho = 0.15625000000000008
n = 5, rho = 0.15625
n = 6, rho = 0.15625000000000006
n = 7, rho = 0.15625000000000022
n = 8, rho = 0.15624999999999983
n = 9, rho = 0.1562500000000002
n = 10, rho = 0.15625
n = 11, rho = 0.15625000000000003
n = 12, rho = 0.15624999999999972
n = 13, rho = 0.1562499999999998
n = 14, rho = 0.1562500000000004
n = 15, rho = 0.15625000000000083
n = 16, rho = 0.15625000000000047
n = 17, rho = 0.15625000000000006
n = 18, rho = 0.15625000000000033
n = 19, rho = 0.15625000000000047
n = 20, rho = 0.1562499999999999
n = 21, rho = 0.15625000000000025
n = 22, rho = 0.15625000000000044
n = 23, rho = 0.1562499999999998
n = 24, rho = 0.1562499999999997
n = 25, rho = 0.15625000000000036
n = 26, rho = 0.1562500000000001
n = 27, rho = 0.15624999999999997
n = 28, rho = 0.15624999999999983
n = 29, rho = 0.15625000000000092
n = 30, rho = 0.15624999999999978
n = 31, rho = 0.15625000000000053
n = 32, rho = 0.15625000000000047
n = 33, rho = 0.15

In [200]:
def jacobi_method(A, b, rho, typed=1, stop=1):
    iterations = 0
    if typed == 1:
        x = np.zeros_like(b)
    else:
        x = np.array([100 if i % 2 == 0 else -100 for i in range(b.shape[0])])
        
    D = np.diag(A)
    R = A - np.diagflat(D)
    while True:
        x_new = (b - R @ x) / D
        iterations += 1
        if stop == 1:
            if np.linalg.norm(x_new - x) < rho:
                break
        else:
            if np.linalg.norm(A @ x_new - b) < rho:
                break
        x = x_new
            
    return x_new, iterations

In [201]:
rhos = [1e-1, 1e-3, 1e-5, 1e-7, 1e-9, 1e-11]

In [202]:
def generate_iterations_table(typed, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    n_values = list(range(5, 431, 25)) + [500,1000]

    for n in n_values:
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            _, iterations = jacobi_method(A, b, rho=rho, typed=typed, stop=stop)
            row += f"& {iterations} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if typed == 1 and stop == 1:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [0, 0, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    elif typed == 1 and stop == 2:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [0, 0, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    elif typed == 2 and stop == 1:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [100, -100, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [100, -100, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [203]:
def generate_difference_table(typed, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    n_values = list(range(5, 431, 25)) + [500,1000]

    for n in n_values:
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            x, _ = jacobi_method(A, b, rho=rho, typed=typed, stop=stop)
            diff = difference(b, A @ x)
            row += f"& {diff:.8f} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if typed == 1 and stop == 1:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [0, 0, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    elif typed == 1 and stop == 2:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [0, 0, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    elif typed == 2 and stop == 1:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [100, -100, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [100, -100, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [204]:
def generate_execution_time_table(typed, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    n_values = list(range(5, 431, 25)) + [500,1000]
    
    for n in n_values:
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            exec_time = timeit.timeit(lambda: jacobi_method(A, b, rho=rho, typed=typed, stop=stop), number=1)
            row += f"& {exec_time:.8f} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if typed == 1 and stop == 1:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [0, 0, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    elif typed == 1 and stop == 2:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [0, 0, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    elif typed == 2 and stop == 1:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [100, -100, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [100, -100, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [205]:
print(generate_iterations_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 2 & 4 & 7 & 9 & 12 & 14 \\ \hline
30 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
55 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
80 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
105 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
130 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
155 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
180 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
205 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
230 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
255 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
280 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
305 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
330 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
355 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
380 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
405 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
430 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
500 & 3 & 

In [206]:
print(generate_difference_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.03203668 & 0.00078881 & 0.00000267 & 0.00000007 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.00586995 & 0.00013075 & 0.00000240 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.00697123 & 0.00014847 & 0.00000232 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.00804614 & 0.00015937 & 0.00000228 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00883454 & 0.00016729 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.00945857 & 0.00017350 & 0.00000233 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.00997413 & 0.00017863 & 0.00000235 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.01041403 & 0.00018299 & 0.00000237 & 0.00000008 & 0.00000000 & 0.00000000 \\ \

In [207]:
print(generate_execution_time_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00009210 & 0.00005160 & 0.00006600 & 0.00007750 & 0.00011420 & 0.00011740 \\ \hline
30 & 0.00006130 & 0.00007020 & 0.00008140 & 0.00010690 & 0.00010090 & 0.00018140 \\ \hline
55 & 0.00008040 & 0.00008380 & 0.00012730 & 0.00012350 & 0.00014560 & 0.00015150 \\ \hline
80 & 0.00010360 & 0.00007020 & 0.00007900 & 0.00009630 & 0.00011200 & 0.00014270 \\ \hline
105 & 0.00052800 & 0.00058640 & 0.00076610 & 0.00101250 & 0.00130850 & 0.00137630 \\ \hline
130 & 0.00040690 & 0.00060770 & 0.00078030 & 0.00086070 & 0.00106890 & 0.00105010 \\ \hline
155 & 0.00040520 & 0.00061880 & 0.00065100 & 0.00082430 & 0.00127240 & 0.00106430 \\ \hline
180 & 0.00041940 & 0.00045040 & 0.00056050 & 0.00082300 & 0.00118220 & 0.00145290 \\ \

In [208]:
print(generate_iterations_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 2 & 4 & 7 & 9 & 12 & 14 \\ \hline
30 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
55 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
80 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
105 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
130 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
155 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
180 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
205 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
230 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
255 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
280 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
305 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
330 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
355 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
380 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
405 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
430 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
500 & 3 & 

In [209]:
print(generate_difference_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.03203668 & 0.00078881 & 0.00000267 & 0.00000007 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.00586995 & 0.00013075 & 0.00000240 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.00697123 & 0.00014847 & 0.00000232 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.00804614 & 0.00015937 & 0.00000228 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00883454 & 0.00016729 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.00945857 & 0.00017350 & 0.00000233 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.00997413 & 0.00017863 & 0.00000235 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.01041403 & 0.00018299 & 0.00000237 & 0.00000008 & 0.00000000 & 0.00000000 \\ \

In [210]:
print(generate_execution_time_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00014410 & 0.00007020 & 0.00011870 & 0.00012360 & 0.00014130 & 0.00022070 \\ \hline
30 & 0.00008000 & 0.00010600 & 0.00010020 & 0.00013660 & 0.00016080 & 0.00055410 \\ \hline
55 & 0.00008290 & 0.00007140 & 0.00009890 & 0.00010560 & 0.00012770 & 0.00014560 \\ \hline
80 & 0.00006380 & 0.00007930 & 0.00009310 & 0.00010950 & 0.00013930 & 0.00015570 \\ \hline
105 & 0.00060800 & 0.00084430 & 0.00104210 & 0.00131400 & 0.00201890 & 0.00244670 \\ \hline
130 & 0.00069220 & 0.00104330 & 0.00143000 & 0.00173160 & 0.00236800 & 0.00260560 \\ \hline
155 & 0.00059620 & 0.00086270 & 0.00142270 & 0.00196770 & 0.00268400 & 0.00320050 \\ \hline
180 & 0.00075960 & 0.00086190 & 0.00131470 & 0.00189710 & 0.00250110 & 0.00300020 \\ \

In [211]:
print(generate_iterations_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 5 & 8 & 10 & 13 & 15 & 18 \\ \hline
30 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
55 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
80 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
105 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
130 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
155 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
180 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
205 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
230 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
255 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
280 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
305 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
330 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
355 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
380 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
405 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
430 & 6 & 8 & 10

In [212]:
print(generate_difference_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.06864039 & 0.00033275 & 0.00000748 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.01973649 & 0.00032769 & 0.00000680 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.02118610 & 0.00032018 & 0.00000650 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.02204760 & 0.00031516 & 0.00000631 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.02266334 & 0.00031142 & 0.00000617 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.02314150 & 0.00030842 & 0.00000607 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.02353316 & 0.00030594 & 0.00000598 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.02386429 & 0.00030380 & 0.00000590 & 0.00000003 & 0.00000000 & 0.00000000 \\ \

In [213]:
print(generate_execution_time_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00024760 & 0.00016430 & 0.00024490 & 0.00029710 & 0.00012370 & 0.00013360 \\ \hline
30 & 0.00006920 & 0.00008330 & 0.00013200 & 0.00016430 & 0.00016640 & 0.00020560 \\ \hline
55 & 0.00009280 & 0.00008580 & 0.00009900 & 0.00011980 & 0.00013240 & 0.00014980 \\ \hline
80 & 0.00009510 & 0.00026230 & 0.00011980 & 0.00013910 & 0.00014050 & 0.00015970 \\ \hline
105 & 0.00061140 & 0.00094130 & 0.00109620 & 0.00135630 & 0.00168660 & 0.00187600 \\ \hline
130 & 0.00081950 & 0.00115010 & 0.00115460 & 0.00130690 & 0.00139340 & 0.00166080 \\ \hline
155 & 0.00064930 & 0.00072190 & 0.00083420 & 0.00119960 & 0.00129760 & 0.00138950 \\ \hline
180 & 0.00068510 & 0.00071460 & 0.00126160 & 0.00117270 & 0.00136470 & 0.00161390 \\ \

In [214]:
print(generate_iterations_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
30 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
55 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
80 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
105 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
130 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
155 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
180 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
205 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
230 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
255 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
280 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
305 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
330 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
355 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
380 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
405 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
430 & 6 & 8 & 10

In [215]:
print(generate_difference_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.01509448 & 0.00033275 & 0.00000748 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.01973649 & 0.00032769 & 0.00000680 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.02118610 & 0.00032018 & 0.00000650 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.02204760 & 0.00031516 & 0.00000631 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.02266334 & 0.00031142 & 0.00000617 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.02314150 & 0.00030842 & 0.00000607 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.02353316 & 0.00030594 & 0.00000598 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.02386429 & 0.00030380 & 0.00000590 & 0.00000003 & 0.00000000 & 0.00000000 \\ \

In [216]:
print(generate_execution_time_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00016310 & 0.00011320 & 0.00015910 & 0.00018170 & 0.00017010 & 0.00019360 \\ \hline
30 & 0.00013330 & 0.00011440 & 0.00013000 & 0.00015990 & 0.00021430 & 0.00025650 \\ \hline
55 & 0.00014070 & 0.00013450 & 0.00029370 & 0.00025160 & 0.00019650 & 0.00018870 \\ \hline
80 & 0.00017850 & 0.00011180 & 0.00013590 & 0.00015280 & 0.00017270 & 0.00020530 \\ \hline
105 & 0.00098450 & 0.00119320 & 0.00145590 & 0.00190350 & 0.00268290 & 0.00269700 \\ \hline
130 & 0.00122940 & 0.00134110 & 0.00179720 & 0.00208580 & 0.00227880 & 0.00326150 \\ \hline
155 & 0.00118050 & 0.00169760 & 0.00185640 & 0.00233120 & 0.00293470 & 0.00279820 \\ \hline
180 & 0.00105670 & 0.00123510 & 0.00173280 & 0.00199690 & 0.00229980 & 0.00283800 \\ \

In [217]:
def generate_latex_table_radius():
    n_values = list(range(5, 431, 25)) + [500,1000]
    table_rows = []

    for n in n_values:
        A, _ = generate_matrix_and_vector(n)
        M = jacobi_iteration_matrix(A)
        radius = spectral_radius(M)
        table_rows.append((n, radius))

    latex_code = "\\begin{table}[h!]\n\\centering\n\\begin{tabular}{|c|c|}\n\\hline\nn & Promień spektralny \\\\ \\hline\n"

    for n, radius in table_rows:
        latex_code += f"{n} & {radius:.6f} \\\\ \\hline\n"

    latex_code += "\\end{tabular}\n\\caption{Promienie spektralne dla różnych wartości n}\n\\end{table}"

    return latex_code

In [218]:
print(generate_latex_table_radius())

\begin{table}[h!]
\centering
\begin{tabular}{|c|c|}
\hline
n & Promień spektralny \\ \hline
5 & 0.156250 \\ \hline
30 & 0.156250 \\ \hline
55 & 0.156250 \\ \hline
80 & 0.156250 \\ \hline
105 & 0.156250 \\ \hline
130 & 0.156250 \\ \hline
155 & 0.156250 \\ \hline
180 & 0.156250 \\ \hline
205 & 0.156250 \\ \hline
230 & 0.156250 \\ \hline
255 & 0.156250 \\ \hline
280 & 0.156250 \\ \hline
305 & 0.156250 \\ \hline
330 & 0.156250 \\ \hline
355 & 0.156250 \\ \hline
380 & 0.156250 \\ \hline
405 & 0.156250 \\ \hline
430 & 0.156250 \\ \hline
500 & 0.156250 \\ \hline
1000 & 0.156250 \\ \hline
\end{tabular}
\caption{Promienie spektralne dla różnych wartości n}
\end{table}
